In [75]:
!pip install google-colab
!pip install google-auth google-auth-oauthlib google-auth-httplib2

  Using cached google-colab-1.0.0.tar.gz (72 kB)
  Preparing metadata (setup.py) ... done
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [324 lines of output]
      /var/tmp/pip-install-dphqwrv9/pandas_5b43a7107dab46f3991504eaaa2b0846/setup.py:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        import pkg_resources
      /var/tmp/pip-install-dphqwrv9/pandas_5b43a7107dab46f3991504eaaa2b0846/setup.py:50: DeprecationWarning: distutils Version classes are deprecated. Use 

In [76]:
PROJECT_ID = "platformwenda-1571901528109"
BUCKET_NAME = "gs://customtraining88"
REGION = "us-central1"
SERVICE_ACCOUNT = "vgm-test@platformwenda-1571901528109.iam.gserviceaccount.com"

In [40]:
#!gcloud auth login

In [41]:
#from google.colab import auth

#auth.authenticate_user()


In [42]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PyTorch document AI Model Single GPU using Vertex Training with Custom Container

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/community-content/pytorch_image_classification_single_gpu_with_vertex_sdk_and_torchserve/vertex_training_with_custom_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Setup

In [77]:
PROJECT_ID = "platformwenda-1571901528109"
BUCKET_NAME = "gs://customtraining88"
REGION = "us-central1"
SERVICE_ACCOUNT = "vgm-test@platformwenda-1571901528109.iam.gserviceaccount.com"
content_name = "documentaicontainer1.0"

## Local Training

In [78]:
! ls trainer

test  train


In [79]:
! cat requirements.txt

pip==23.1.2
accelerate==0.21.0
git+https://github.com/huggingface/transformers.git
datasets==2.16.1
pandas
typing-extensions==4.3.0
shapely
scikit-image
imgaug
pyclipper
lmdb
tqdm
numpy
visualdl
rapidfuzz
opencv-python==4.6.0.66
opencv-contrib-python==4.6.0.66
cython
lxml
premailer
openpyxl
attrdict
Polygon3
lanms-neo==1.0.2
PyMuPDF<1.21.0
PyYAML==6.0
fpdf==1.7.2
pdfkit==1.0.0
wkhtmltopdf==0.2
extract-msg==0.39.0
msgpack==1.0.4
pytesseract==0.3.9
seqeval==1.2.2
pdf2image==1.16.0
lxml==4.9.1
opencv-python-headless==4.6.0.66
google-cloud-vision==3.4.4
torch
torchvision
tabulate
bs4
imgkit
google-cloud-storage

In [80]:
! pip install -r requirements.txt

  Cloning https://github.com/huggingface/transformers.git to /var/tmp/pip-req-build-jgoafar2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /var/tmp/pip-req-build-jgoafar2
  Resolved https://github.com/huggingface/transformers.git to commit 3f69f415adcbdaedec154ba8eac220ef3276975d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [81]:
#! cat task.py

In [83]:
#%run task.py  --local-mode

In [84]:
#! ls ./tmp

In [85]:
#! rm -rf ./tmp

## Vertex Training using Vertex SDK and Custom Container

### Build Custom Container

In [86]:
hostname = "gcr.io"
image_name_train = content_name
tag = "latest"

custom_container_image_uri_train = f"{hostname}/{PROJECT_ID}/{image_name_train}:{tag}"
print(custom_container_image_uri_train)

gcr.io/platformwenda-1571901528109/documentaicontainer1.0:latest


In [87]:
#! cd trainer && docker build -t $custom_container_image_uri_train -f Dockerfile .
!docker build -t $custom_container_image_uri_train -f Dockerfile .

Sending build context to Docker daemon  1.727GB
Step 1/9 : FROM pytorch/pytorch:1.8.1-cuda11.1-cudnn8-runtime
 ---> 5ffed6c83695
Step 2/9 : RUN apt-get update &&     apt-get install -y --no-install-recommends         make         gcc         libc-dev         musl-dev         ffmpeg         libsm6         libxext6         tesseract-ocr         libtesseract-dev         libreoffice-common         wkhtmltopdf         libreoffice-writer     && apt-get clean     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> f424a0506d5a
Step 3/9 : RUN apt-get update &&     apt-get install -y --no-install-recommends         git         build-essential     && apt-get clean     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 4d71248df555
Step 4/9 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> cd2123547d23
Step 5/9 : WORKDIR /trainer
 ---> Using cache
 ---> 86372aec3918
Step 6/9 : COPY requirements.txt .
 ---> Using cache
 ---> 486ca27fba91
Step 7/9 : RUN python -m pip install --upg

In [88]:
#! docker run --rm $custom_container_image_uri_train --epochs 2 --local-mode
! docker run --rm $custom_container_image_uri_train --local-mode

preprocessor_config.json: 100%|██████████| 275/275 [00:00<00:00, 65.0kB/s]
tokenizer_config.json: 100%|██████████| 1.14k/1.14k [00:00<00:00, 269kB/s]
config.json: 100%|██████████| 856/856 [00:00<00:00, 512kB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 14.3MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 36.9MB/s]
/opt/conda/lib/python3.8/site-packages/datasets/load.py:922: FutureWarning: The repository for dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at dataset.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 33 examples [00:00, 235.58 examples/s]
Generating test split: 0 examples [00:00, ? examples/s]dataset/train/images/EFLOW_20220104053707NN_IMAGE20220104_06360433060.jpg
dataset/train/images/

In [89]:
#! docker push $custom_container_image_uri_train
! docker push $custom_container_image_uri_train

The push refers to repository [gcr.io/platformwenda-1571901528109/documentaicontainer1.0]

b685f71f: Preparing 
8d4fe012: Preparing 
bb008b26: Preparing 
3213118b: Preparing 
443e4314: Preparing 
e9c44a93: Preparing 
62e73fa9: Preparing 
491659cb: Preparing 
dc413928: Preparing 
ad8f2cae: Preparing 
581dbc3c: Preparing 
685f71f: Pushed   1.727GB/1.727GBlatest: digest: sha256:190236a390f82a663de64b9b4a9a62a34adeed0cc3b04055cf838e686d27642f size: 2842


In [90]:
#! gcloud container images list --repository $hostname/$PROJECT_ID
! gcloud container images list --repository $hostname/$PROJECT_ID

NAME
gcr.io/platformwenda-1571901528109/aicontainer
gcr.io/platformwenda-1571901528109/cont-torchserve
gcr.io/platformwenda-1571901528109/documentaicontainer1.0
gcr.io/platformwenda-1571901528109/documentaimodelcontainer
gcr.io/platformwenda-1571901528109/pt-img-cls-gpu-cust-cont-torchserve
gcr.io/platformwenda-1571901528109/pt-img-cls-gpu-cust-cont-torchserve-antandbee
gcr.io/platformwenda-1571901528109/pytorchaicontainer


### Initialize Vertex SDK

In [91]:
#! pip install -r requirements.txt

In [92]:
from google.cloud import aiplatform

aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_NAME,
    location=REGION,
)

### Create a Vertex Tensorboard Instance

In [ ]:
tensorboard = aiplatform.Tensorboard.create(
    display_name=content_name,
)

#### Option: Use a Previously Created Vertex Tensorboard Instance

```
tensorboard_name = "Your Tensorboard Resource Name or Tensorboard ID"
tensorboard = aiplatform.Tensorboard(tensorboard_name=tensorboard_name)
```

### Run a Vertex SDK CustomContainerTrainingJob

In [ ]:
display_name = content_name
gcs_output_uri_prefix = f"{BUCKET_NAME}/{display_name}"

machine_type = "n1-standard-4"
accelerator_count = 1
accelerator_type = "NVIDIA_TESLA_K80"

'''
container_args = [
    "--batch-size",
    "256",
    "--epochs",
    "100",
]
'''

In [ ]:
custom_container_training_job = aiplatform.CustomContainerTrainingJob(
    display_name=display_name,
    container_uri=custom_container_image_uri_train,
)

In [ ]:
custom_container_training_job.run(
    #args=container_args,
    base_output_dir=gcs_output_uri_prefix,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    tensorboard=tensorboard.resource_name,
    service_account=SERVICE_ACCOUNT
)

In [ ]:
print(f"Custom Training Job Name: {custom_container_training_job.resource_name}")
print(f"GCS Output URI Prefix: {gcs_output_uri_prefix}")

### Training Artifact

In [ ]:
! gsutil ls $gcs_output_uri_prefix